In [ ]:
#default_exp utils.replicates

In [ ]:
# export
import numpy as np
from einops import rearrange
from itertools import combinations, permutations
from montlake.statistics.basispursuit import get_basispursuit_losses

def get_detected_values2d(subset, supports,nreps):
    
    detected_values = np.zeros((subset.shape[0],subset.shape[0]))
    nreps = len(list(supports.keys()))
    for r in range(nreps):
        i1 = np.where(subset == int(supports[r][0]))[0]
        i2 = np.where(subset == int(supports[r][1]))[0]
        detected_values[i1,i2] += 1
        detected_values[i2,i1] += 1
    
    detected_values = np.asarray(np.where(detected_values > 0))
    return(detected_values)

def get_support_indices2d(sup_set, sup_sel,nreps):
    
    #toplot = np.zeros(np.repeat(sup_set.shape[0], d ))
    toplot = np.zeros((sup_set.shape[0],sup_set.shape[0]))
    for r in range(nreps):
        i1 = np.where(sup_set == sup_sel[r][0])[0]
        i2 = np.where(sup_set == sup_sel[r][1])[0]
        toplot[i1,i2] += 1
        toplot[i2,i1] += 1   
    return(toplot)

# def get_supports(nreps,p,d, selected_functions):
#     for r in range(nreps):
#         subset_associator = np.asarray(list(enumerate(selected_functions_lm2[r])))
#         sub_sets = list(combinations(subset_associator[:,0], d))
#         full_sets = list(combinations(subset_associator[:,1], d))
#         basispursuit_losses_subset = get_basispursuit_losses(replicates[r].dg_M[:,:,selected_functions_lm2[r]],replicates[r].dphispectral_M_normalized)
#         for s in range(len(full_sets)):
#             ols_norms[r][full_sets[s]] = basispursuit_losses_subset[sub_sets[s]]
#         ind = np.unravel_index(np.nanargmin(ols_norms[r], axis=None), ols_norms[r].shape)
#         toadd = list(permutations(ind,2))
#         for s in range(len(toadd)):
#             supports[toadd[s]] += 1    

def supportlist_to_tensor(supports,p ):
    d = supports.shape[1]
    nreps = supports.shape[0]
    support_tensor = np.zeros(np.repeat(p,d))
    for r in range(nreps):
        toadd = np.asarray(list(permutations(supports[r],d)))
        for s in range(len(toadd)):
            support_tensor[tuple(np.asarray(toadd[s], dtype = int))] += 1
    return(support_tensor)

def get_supports_lasso(replicates, p,d):
    
    nreps = len(list(replicates.keys()))
    supports = np.zeros((nreps, d))
    for r in range(nreps):
        supports[r] = np.where(np.linalg.norm(replicates[r].results[1][replicates[r].results[0]], axis = tuple([0,2])) > 0)[0]
    support_tensor = supportlist_to_tensor(supports,p )
    return(support_tensor,supports)

def get_supports_brute(replicates,nreps,p,d,selected_functions_lm2):
    
    ols_norms= np.zeros(tuple(np.concatenate([[nreps], np.repeat(p,d)])))
    supports = np.zeros((nreps, d), dtype = int)
    ols_norms[:] = np.nan
    for r in range(nreps):
        subset_associator = np.asarray(list(enumerate(selected_functions_lm2[r])))
        sub_sets = list(combinations(subset_associator[:,0], d))
        full_sets = list(combinations(subset_associator[:,1], d))
        basispursuit_losses_subset = get_basispursuit_losses(replicates[r].dg_M[:,:,selected_functions_lm2[r]],replicates[r].dphispectral_M_normalized)
        for s in range(len(full_sets)):
            ols_norms[r][full_sets[s]] = basispursuit_losses_subset[sub_sets[s]]
        supports[r] = np.unravel_index(np.nanargmin(ols_norms[r], axis=None), ols_norms[r].shape)
    support_tensor = supportlist_to_tensor(supports,p )
    return(support_tensor, supports)

def get_supports_brute_tslasso(replicates,nreps,p,d,selected_functions_lm2):
    
    nsel = replicates[0].dg_M.shape[0]
    ols_norms= np.zeros(tuple(np.concatenate([[nreps], np.repeat(p,d)])))
    supports = np.zeros((nreps, d), dtype = int)
    ols_norms[:] = np.nan
    for r in range(nreps):
        subset_associator = np.asarray(list(enumerate(selected_functions_lm2[r])))
        sub_sets = list(combinations(subset_associator[:,0], d))
        full_sets = list(combinations(subset_associator[:,1], d))
        basispursuit_losses_subset = get_basispursuit_losses(replicates[r].dg_M[:,:,selected_functions_lm2[r]],np.asarray([np.identity(d) for i in range(nsel)]))
        for s in range(len(full_sets)):
            ols_norms[r][full_sets[s]] = basispursuit_losses_subset[sub_sets[s]]
        supports[r] = np.unravel_index(np.nanargmin(ols_norms[r], axis=None), ols_norms[r].shape)
    support_tensor = supportlist_to_tensor(supports,p )
    return(support_tensor,supports)



class Replicate():

    def __init__(self, nsel = None, n = None, selected_points = None):

        self.nsel = nsel
        if selected_points is not None:
            self.selected_points = selected_points
        else:
            self.selected_points = np.random.choice(list(range(n)), nsel, replace=False)


    def get_ordered_axes(self):
        replicate = self
        cs = rearrange(np.asarray(list(replicate.results[1].values())), 'l n p m -> l m n p')
        xaxis = np.asarray(np.asarray(list(replicate.results[1].keys())))
        xaxis_reorder = xaxis[xaxis.argsort()]
        cs_reorder = cs[xaxis.argsort()]
        xaxis_reorder = xaxis[xaxis.argsort()]
        replicate.cs = cs
        replicate.cs_reorder = cs_reorder
        replicate.xaxis_reorder = xaxis_reorder
        replicate.xaxis = xaxis

    def get_selection_lambda(self):

        replicate = self
        lambdas = np.asarray(list(replicate.results[1].keys()))
        lambdas.sort()
        sel_l = np.where(lambdas ==  replicate.results[0])[0][0]
        return(sel_l)
